In [2]:
## Importando bibliotecas essenciais de DS
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import random

##### O problema do negócio era simples.
Temos um dataset de caminhões nos quais se classificam caminhões em 'neg' (não deram defeito) e 'pos' (deram defeito).
A missão, então, é simples, prever, a partir das 170 covariadas do dataset quais caminhões vão dar defeito ou não.

O problema é que 170 colunas é perfeito para, de cara termos um problema de overfitting.
Pior que isto, no entanto, é que as colunas estão todas codificadas, ou seja, não consigo inferir sua importância através de alguma análise teórica ou intuitiva da situação.

Isso fez desta modelagem especialmente desafiadora, porque tive que aplicar técnicas específicas para reduzir a dimensionalidade do dataset, remover colunas com muita colinearidade, ou então pouco influentes, usando, inclusive, de técnica como PCA, das quais nunca tinha feito uso.

Enfim, abaixo, a cada passo, estarei comentando com mais profundidade cada etapa essencial

In [5]:
# Estabelecendo uma seed para garantir reprodutibilidade
random.seed(1603)

In [3]:
## lendo o dataset 'previous year', que deve servir como base de nossa projeção
df_previous = pd.read_csv('C:/Users/jpzam/Desktop/Case_study_selection/air_system_previous_years.csv')

df_previous.head()

,class,aa_000,ab_000,ac_000,ad_000,ae_000,af_000,ag_000,ag_001,ag_002,...,ee_002,ee_003,ee_004,ee_005,ee_006,ee_007,ee_008,ee_009,ef_000,eg_000
0,neg,76698,na,2130706438,280,0,0,0,0,0,...,1240520,493384,721044,469792,339156,157956,73224,0,0,0
1,neg,33058,na,0,na,0,0,0,0,0,...,421400,178064,293306,245416,133654,81140,97576,1500,0,0
2,neg,41040,na,228,100,0,0,0,0,0,...,277378,159812,423992,409564,320746,158022,95128,514,0,0
3,neg,12,0,70,66,0,10,0,0,0,...,240,46,58,44,10,0,0,0,4,32
4,neg,60874,na,1368,458,0,0,0,0,0,...,622012,229790,405298,347188,286954,311560,433954,1218,0,0


In [6]:
df_previous['class'].unique()

array(['neg', 'pos'], dtype=object)

In [105]:
## Sumarização breve
print(df_previous.columns)

print(df_previous.dtypes)

Index(['class', 'aa_000', 'ab_000', 'ac_000', 'ad_000', 'ae_000', 'af_000',
       'ag_000', 'ag_001', 'ag_002',
       ...
       'ee_002', 'ee_003', 'ee_004', 'ee_005', 'ee_006', 'ee_007', 'ee_008',
       'ee_009', 'ef_000', 'eg_000'],
      dtype='object', length=171)
class     object
aa_000     int64
ab_000    object
ac_000    object
ad_000    object
           ...  
ee_007    object
ee_008    object
ee_009    object
ef_000    object
eg_000    object
Length: 171, dtype: object


In [106]:
# Aferindo a presença de NAs na coluna 'class', que estão como 'na', em string.
df_previous[df_previous['class'] == 'na'].value_counts()

Series([], dtype: int64)

In [111]:
## Substituindos os 'na' por valores NaN

df_previous = df_previous.replace('na', np.nan)

df_previous.head()

,class,aa_000,ab_000,ac_000,ad_000,ae_000,af_000,ag_000,ag_001,ag_002,...,ee_002,ee_003,ee_004,ee_005,ee_006,ee_007,ee_008,ee_009,ef_000,eg_000
0,neg,76698,NaN,2130706438,280,0,0,0,0,0,...,1240520,493384,721044,469792,339156,157956,73224,0,0,0
1,neg,33058,NaN,0,NaN,0,0,0,0,0,...,421400,178064,293306,245416,133654,81140,97576,1500,0,0
2,neg,41040,NaN,228,100,0,0,0,0,0,...,277378,159812,423992,409564,320746,158022,95128,514,0,0
3,neg,12,0,70,66,0,10,0,0,0,...,240,46,58,44,10,0,0,0,4,32
4,neg,60874,NaN,1368,458,0,0,0,0,0,...,622012,229790,405298,347188,286954,311560,433954,1218,0,0


In [123]:
## Conferência para ver a quantidade de NaN's no dataset.
df_previous.isnull().sum().sort_values(ascending = False)

br_000    49264
bq_000    48722
bp_000    47740
bo_000    46333
ab_000    46329
          ...  
cj_000      338
ci_000      338
bt_000      167
aa_000        0
class         0
Length: 171, dtype: int64

In [100]:
## Tendo em vista que as colunas são, por completo, numéricas, resolvi substituir os NaNs pela moda de cada coluna

# Loop para substituir os NAs pela moda.
for col in df_previous.columns:
    if df_previous[col].isna().sum() > 0:
        moda = df_previous[col].mode()[0]
        df_previous[col] = df_previous[col].fillna(moda)

print(df_previous.isna().sum()) # E conferência final

class     0
aa_000    0
ab_000    0
ac_000    0
ad_000    0
         ..
ee_007    0
ee_008    0
ee_009    0
ef_000    0
eg_000    0
Length: 171, dtype: int64


In [49]:
## Mudando as variáveis numéricas para 'float64'.

df_previous.iloc[:, 1:] = df_previous.iloc[:, 1:].astype(float)

df_previous.dtypes


C:\Users\jpzam\AppData\Local\Temp\ipykernel_10920\3185615704.py:3: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df_previous.iloc[:, 1:] = df_previous.iloc[:, 1:].astype(float)


class      object
aa_000    float64
ab_000    float64
ac_000    float64
ad_000    float64
           ...   
ee_007    float64
ee_008    float64
ee_009    float64
ef_000    float64
eg_000    float64
Length: 171, dtype: object

In [50]:
df_previous.head()

,class,aa_000,ab_000,ac_000,ad_000,ae_000,af_000,ag_000,ag_001,ag_002,...,ee_002,ee_003,ee_004,ee_005,ee_006,ee_007,ee_008,ee_009,ef_000,eg_000
0,neg,76698.0,0.0,2.130706e+09,280.0,0.0,0.0,0.0,0.0,0.0,...,1240520.0,493384.0,721044.0,469792.0,339156.0,157956.0,73224.0,0.0,0.0,0.0
1,neg,33058.0,0.0,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,...,421400.0,178064.0,293306.0,245416.0,133654.0,81140.0,97576.0,1500.0,0.0,0.0
2,neg,41040.0,0.0,2.280000e+02,100.0,0.0,0.0,0.0,0.0,0.0,...,277378.0,159812.0,423992.0,409564.0,320746.0,158022.0,95128.0,514.0,0.0,0.0
3,neg,12.0,0.0,7.000000e+01,66.0,0.0,10.0,0.0,0.0,0.0,...,240.0,46.0,58.0,44.0,10.0,0.0,0.0,0.0,4.0,32.0
4,neg,60874.0,0.0,1.368000e+03,458.0,0.0,0.0,0.0,0.0,0.0,...,622012.0,229790.0,405298.0,347188.0,286954.0,311560.0,433954.0,1218.0,0.0,0.0


In [51]:
## Transformando 'classe' em uma coluna booleana

df_previous['class'] = df_previous['class'] != 'neg' ## neg = 0;False ;; pos = 1; True
df_previous

df_previous.head()

,class,aa_000,ab_000,ac_000,ad_000,ae_000,af_000,ag_000,ag_001,ag_002,...,ee_002,ee_003,ee_004,ee_005,ee_006,ee_007,ee_008,ee_009,ef_000,eg_000
0,False,76698.0,0.0,2.130706e+09,280.0,0.0,0.0,0.0,0.0,0.0,...,1240520.0,493384.0,721044.0,469792.0,339156.0,157956.0,73224.0,0.0,0.0,0.0
1,False,33058.0,0.0,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,...,421400.0,178064.0,293306.0,245416.0,133654.0,81140.0,97576.0,1500.0,0.0,0.0
2,False,41040.0,0.0,2.280000e+02,100.0,0.0,0.0,0.0,0.0,0.0,...,277378.0,159812.0,423992.0,409564.0,320746.0,158022.0,95128.0,514.0,0.0,0.0
3,False,12.0,0.0,7.000000e+01,66.0,0.0,10.0,0.0,0.0,0.0,...,240.0,46.0,58.0,44.0,10.0,0.0,0.0,0.0,4.0,32.0
4,False,60874.0,0.0,1.368000e+03,458.0,0.0,0.0,0.0,0.0,0.0,...,622012.0,229790.0,405298.0,347188.0,286954.0,311560.0,433954.0,1218.0,0.0,0.0


Feita a limpeza e tratamento dos dados nas linhas acima, aqui, abaixo, importo os pacotes essenciais para a construção do modelo.

Tendo em vista a pergunta a ser respondida, optei por um modelo de regressão logística, para o qual usarei a biblioteca sickit-learn.

Antes de tudo, porém, precisava reduzir a dimensionalidade desse dataset e e selecionar as melhores variáveis para a predição.

In [52]:
from scipy.stats import pointbiserialr
from sklearn import linear_model

In [53]:
## Calculando as correlações com 'classe' para cada coluna
    ## Neste momento, pensei que isto talvez bastaria para que eu pudesse encontrar alguma luz que me apontasse quais variáveis usar
correlacoes = []

for col in df_previous.columns:
    if col != 'class':
        correlacao = pointbiserialr(df_previous['class'], df_previous[col])
        correlacoes.append((col, correlacao[0], correlacao[1])) # Coluna, correlação, p-valor

print(correlacoes)

[('aa_000', 0.5369783925131109, 0.0), ('ab_000', 0.015084381201274314, 0.00021985482974012596), ('ac_000', -0.05099634473091605, 7.524064561691334e-36), ('ad_000', -0.0005297612434709985, 0.8967549032172947), ('ae_000', 0.005541072015891219, 0.17469796167472249), ('af_000', 0.016249922517591543, 6.874682388440666e-05), ('ag_000', 0.01235259091904953, 0.002479706370481349), ('ag_001', 0.19108385741369696, 0.0), ('ag_002', 0.3399556782143777, 0.0), ('ag_003', 0.43305030605979766, 0.0), ('ag_004', 0.3716224837308859, 0.0), ('ag_005', 0.44817851080983284, 0.0), ('ag_006', 0.36834794713121316, 0.0), ('ag_007', 0.24191004312131892, 0.0), ('ag_008', 0.11300610195605039, 1.0235682569954864e-169), ('ag_009', 0.07208137875681554, 6.098446719681328e-70), ('ah_000', 0.511630935180927, 0.0), ('ai_000', 0.11827308754982259, 8.029574893207054e-186), ('aj_000', 0.023871558015076646, 4.974435179784846e-09), ('ak_000', 0.011752894095135972, 0.003990663732019046), ('al_000', 0.36545199897228575, 0.0), ('

c:\Users\jpzam\anaconda3\lib\site-packages\scipy\stats\_stats_py.py:4424: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(stats.ConstantInputWarning(msg))


In [54]:
## Colocando correlacoes em um dataframe; e ordenando para seleção
corr_df = pd.DataFrame(correlacoes, columns = ['coluna_df', 'correlacao', 'p_valor'])

# Correlações positivas
corr_df = corr_df.sort_values(by = ['p_valor', 'correlacao'], ascending = [True, False]).reset_index(drop = True)
corr_df.head(10)

## Com dá para ver, não foi muito efetivo, não dá para tirar muita coisa daqui. 
    ## Mas parece haver indícios de que muitas colunas, afinal, implicam coisas parecidas para 'class'

,coluna_df,correlacao,p_valor
0,ci_000,0.550049,0.0
1,aa_000,0.536978,0.0
2,bt_000,0.533964,0.0
3,bb_000,0.529501,0.0
4,bv_000,0.528056,0.0
5,bu_000,0.528056,0.0
6,cq_000,0.528056,0.0
7,aq_000,0.518841,0.0
8,bj_000,0.513465,0.0
9,cc_000,0.511886,0.0


In [55]:
# Correlações negativas.
corr_df = corr_df.sort_values(by = ['p_valor', 'correlacao'], ascending = [True, True]).reset_index(drop = True)
corr_df.head(5)

,coluna_df,correlacao,p_valor
0,br_000,-0.274250,0.0
1,bq_000,-0.263856,0.0
2,bp_000,-0.245132,0.0
3,bo_000,-0.222640,0.0
4,bn_000,-0.192862,0.0


In [58]:
## Aqui tentei fazer um teste de variança, para escolher quais colunas eu poderia droppar.
    ## Ao fim do modelo, percebi que este teste não contribuía tanto para a seleção das covariadas e optei por removê-lo por completo.
#from sklearn.feature_selection import VarianceThreshold

# Setting the threshold
#select = VarianceThreshold(threshold = 10) # <- Variance threshold; 1 reduced a fair ammount

#select.fit(df_previous/df_previous.mean()) ## Nomalizing data
#mask = select.get_support()

df_previous_reduzido = df_previous#.loc[:, mask] ## Aqui removi a máscara, deixando-a comentada. 
    ## Para aquele que vê o modelo, caso tenha sugestões de como fazer um melhor uso deste teste, sinta-se a vontade para dar ideias.

print(df_previous.shape)
print(df_previous_reduzido.shape)

(60000, 171)
(60000, 171)


In [59]:
df_test = df_previous_reduzido.iloc[:, 1:]

In [60]:
## Testando colinearidade
from statsmodels.stats.outliers_influence import variance_inflation_factor

vif = pd.DataFrame()

vif["features"] = df_test.columns

vif['fator_vif'] = [variance_inflation_factor(df_test.values, i) for i in range(df_test.shape[1])]

vif

,features,fator_vif
0,aa_000,4.012605e+02
1,ab_000,1.068214e+00
2,ac_000,1.281894e+00
3,ad_000,1.266802e+09
4,ae_000,3.173399e+00
...,...,...
165,ee_007,1.826089e+09
166,ee_008,1.236384e+08
167,ee_009,1.377875e+06
168,ef_000,1.128800e+00


In [61]:
## selecionando aqui os fatores cujos vifs sejam inferiores a 5. A literatura vigente parece indicar que este é um bom threshold.

vif = vif[vif['fator_vif'] <= 5]

list(vif['features'])

['ab_000',
 'ac_000',
 'ae_000',
 'af_000',
 'ag_000',
 'ai_000',
 'aj_000',
 'ak_000',
 'ar_000',
 'as_000',
 'at_000',
 'au_000',
 'av_000',
 'ax_000',
 'bc_000',
 'bd_000',
 'be_000',
 'bf_000',
 'bl_000',
 'bm_000',
 'bs_000',
 'bz_000',
 'ca_000',
 'cg_000',
 'ch_000',
 'cj_000',
 'cl_000',
 'cm_000',
 'cp_000',
 'cr_000',
 'cs_001',
 'cy_000',
 'da_000',
 'db_000',
 'dd_000',
 'de_000',
 'df_000',
 'dg_000',
 'dh_000',
 'di_000',
 'dj_000',
 'dk_000',
 'dq_000',
 'dr_000',
 'dx_000',
 'dy_000',
 'dz_000',
 'ea_000',
 'eb_000',
 'ef_000',
 'eg_000']

In [62]:
## Filtrando aqueles que são inferiores a cinco

df_previous_reduzido.columns.isin(list(vif['features']))

vif_features = vif['features'].tolist()

df_previous_filtered = df_previous_reduzido[['class'] + vif_features]

df_previous_filtered.head()

,class,ab_000,ac_000,ae_000,af_000,ag_000,ai_000,aj_000,ak_000,ar_000,...,dk_000,dq_000,dr_000,dx_000,dy_000,dz_000,ea_000,eb_000,ef_000,eg_000
0,False,0.0,2.130706e+09,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2801180.0,0.0,0.0
1,False,0.0,0.000000e+00,0.0,0.0,0.0,0.0,68.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3477820.0,0.0,0.0
2,False,0.0,2.280000e+02,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1040120.0,0.0,0.0
3,False,0.0,7.000000e+01,0.0,10.0,0.0,0.0,0.0,0.0,0.0,...,0.0,2014.0,370.0,20174.0,44.0,0.0,0.0,0.0,4.0,32.0
4,False,0.0,1.368000e+03,0.0,0.0,0.0,0.0,226.0,0.0,0.0,...,0.0,0.0,0.0,98334.0,27588.0,0.0,0.0,21173050.0,0.0,0.0


In [63]:
## Fazendo agora PCA para reduzir ainda mais a dimensionalidade.
    ## Como interpretabilidade não era, exatamente, o objetivo do modelo, fazer combinações assim não trazia muitos prejuízos.
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
df_previous_std = scaler.fit_transform(df_previous_filtered) ## Aplicando o standard scaler
pca = PCA(n_components = 30)

pc = pca.fit_transform(df_previous_std)

print(pca.explained_variance_ratio_)

# Testing with df_previous; unfiltered.

#df_previous_std = scaler.fit_transform(df_previous)

#pca = PCA()

#pca.fit(df_previous_std)

#print(pca.explained_variance_ratio_)

print(pca.explained_variance_ratio_.cumsum())

[0.11784524 0.04351646 0.03770755 0.03700104 0.03326157 0.03285729
 0.03168051 0.03011558 0.02941565 0.02687478 0.02475232 0.02371396
 0.02258273 0.02244625 0.02187578 0.02146052 0.02057176 0.01987971
 0.01932735 0.01910909 0.01893057 0.01876035 0.01856705 0.01789909
 0.01725618 0.01708348 0.01622566 0.01586558 0.01509111 0.01447534]
[0.11784524 0.1613617  0.19906924 0.23607028 0.26933185 0.30218914
 0.33386965 0.36398523 0.39340089 0.42027566 0.44502799 0.46874194
 0.49132467 0.51377092 0.5356467  0.55710722 0.57767898 0.59755869
 0.61688605 0.63599514 0.65492571 0.67368606 0.69225311 0.7101522
 0.72740837 0.74449186 0.76071752 0.77658309 0.79167421 0.80614954]


In [64]:
print(pc[:, 0]) ## All Rows and First columns

print(pc[:, 29])

[ 0.49490464  0.01048822 -0.5708673  ... -1.14848706  0.42921646
 -0.15611435]
[ 0.21893324  0.01415137 -0.12493176 ... -0.0609056  -0.05427971
 -0.1631982 ]


In [65]:
## Mirando para 80% de explicação do modelo, um valor ok para uma base com tantas covariadas
    ## O objetivo é reduzir para, ao menos, 30.    

df_final = pd.DataFrame(pc, columns = [f'PC{n}' for n in range (0,30)])

df_final = pd.concat([df_previous_filtered['class'], df_final], axis = 1)

df_final

,class,PC0,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,...,PC20,PC21,PC22,PC23,PC24,PC25,PC26,PC27,PC28,PC29
0,False,0.494905,-1.059201,1.062445,-0.535933,0.513086,0.603192,-0.702623,-0.006713,0.822267,...,0.919182,-1.454420,0.012090,-0.280408,-0.299714,-0.409327,-0.391185,-0.257251,0.057018,0.218933
1,False,0.010488,-0.872356,0.875366,-0.679592,0.255803,0.326954,-0.534286,-0.163695,0.307178,...,-0.026207,0.393266,0.024439,-0.097960,-0.135541,-0.101951,0.290486,-0.137042,0.243125,0.014151
2,False,-0.570867,0.098797,-0.087793,-0.064974,0.038725,-0.080620,-0.037610,-0.046919,-0.026122,...,-0.263511,0.362920,0.037765,0.023824,-0.034627,-0.033959,0.019762,0.141659,0.055346,-0.124932
3,False,-0.992462,1.406792,0.032266,1.048060,-0.442415,0.022493,1.237495,-0.708838,-0.285095,...,-0.457199,0.179074,-0.747485,0.381123,-0.111548,0.108816,-0.436770,0.160321,0.130380,0.745097
4,False,0.594239,-1.013507,0.983505,-0.719509,0.618919,0.527497,-0.450578,0.074845,0.596864,...,-0.196619,0.485363,0.092997,-0.130889,-0.002419,0.195094,-0.120048,-0.031165,0.099765,-0.084680
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59995,False,1.701616,-1.637151,1.482476,-0.992811,0.640593,0.961692,-0.976966,0.040775,1.042461,...,-0.027366,0.478255,0.016399,-0.374354,0.202647,0.071044,-0.762190,-0.362003,-0.245749,0.130631
59996,False,-1.094575,0.733573,-0.588224,0.263004,-0.046147,-0.125595,0.306297,0.102874,-0.014849,...,0.834754,-1.682350,-0.296410,-0.029161,-0.186364,-0.340108,-0.278345,0.043041,0.137419,-0.030506
59997,False,-1.148487,0.809949,-0.696847,0.347153,-0.137540,-0.185336,0.355405,0.078095,-0.134206,...,0.847117,-1.694833,-0.261764,-0.043111,-0.186779,-0.350129,-0.346767,0.012415,0.207476,-0.060906
59998,False,0.429216,-0.544701,0.179784,-0.126782,0.384389,0.232559,0.110393,0.070833,-0.069923,...,0.829635,-1.582146,-0.055981,-0.292859,-0.120617,-0.350319,-0.436680,0.276874,-0.029423,-0.054280


In [66]:
# Aqui usei uma técnica que aprendi do DataCamp
    # Forward stepwise selection
    ## Esta técnica vai nos permitir selecionar, a partir do dataset de teste, as variáveis que levar ao melhor AUC possível
    ## Não espero que as 30 variáveis entrem nesta seleção.
from sklearn.metrics import roc_auc_score

# Function to calculate AUC
def auc(vars, explained, df):
    X = df[vars]
    y = df[explained]

    logreg = linear_model.LogisticRegression()
    logreg.fit(X, y)

    preds = logreg.predict_proba(X)[:, 1]
    auc = roc_auc_score(y, preds)
    return(auc)

# Função para determinar melhor variável
def next_best(current_vars, possible_vars, explained, df):
    best_auc = -1
    melhor_var = None

    for v in possible_vars:
        auc_v = auc(current_vars + [v], explained, df)
        if auc_v >= best_auc:
            best_auc = auc_v
            melhor_var = v
    return melhor_var

In [67]:
## Fazer o split de treino e teste.
from sklearn.model_selection import train_test_split

explicativa = df_final.drop('class', axis = 1)
explicada = df_final['class']

explicativas_train, explicativas_test, explicada_train, explicada_test = train_test_split(explicativa, explicada, test_size = 0.3)

train = pd.concat([explicada_train, explicativas_train], axis=1)
test = pd.concat([explicada_test, explicativas_test], axis=1)

In [68]:
vars_atuais = ['PC0'] ## Estabelecendo variável inicial, primeira coluna, mais explicativa.
auc_train = []
auc_test = []

possiveis = train.drop('class', axis = 1).columns.tolist()
explicada = 'class'

for n in range(10): ## Range arbitrário que, caso não suficiente poderia ser expandido.
        ## A função, basicamente, itera diversos fits de regressão logística e vai testando para ver se o AUC em teste aumentou ou caiu.
    best_var = next_best(vars_atuais, possiveis, explicada, train)
    vars_atuais.append(best_var)
    possiveis.remove(best_var)
    
    best_auc_train = auc(vars_atuais, explicada, train)
    best_auc_test = auc(vars_atuais, explicada, test)
    auc_train.append(best_auc_train)
    auc_test.append(best_auc_test)

    if auc_test[n] < auc_test[n-1]:
        print(vars_atuais)
        print(auc_train)
        print(auc_test)
        break
    else:
        pass

## PArece que, com essas cinco vars. conseguimos um AUC bem positivo.

['PC0', 'PC3', 'PC1', 'PC16', 'PC19']
[0.9907293322622629, 0.9946828684835084, 0.9954147494168033, 0.9957205969772847]
[0.9899971116155818, 0.9919253890802397, 0.9928243398392651, 0.9926140531317769]


In [69]:
df_final.head()

,class,PC0,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,...,PC20,PC21,PC22,PC23,PC24,PC25,PC26,PC27,PC28,PC29
0,False,0.494905,-1.059201,1.062445,-0.535933,0.513086,0.603192,-0.702623,-0.006713,0.822267,...,0.919182,-1.454420,0.012090,-0.280408,-0.299714,-0.409327,-0.391185,-0.257251,0.057018,0.218933
1,False,0.010488,-0.872356,0.875366,-0.679592,0.255803,0.326954,-0.534286,-0.163695,0.307178,...,-0.026207,0.393266,0.024439,-0.097960,-0.135541,-0.101951,0.290486,-0.137042,0.243125,0.014151
2,False,-0.570867,0.098797,-0.087793,-0.064974,0.038725,-0.080620,-0.037610,-0.046919,-0.026122,...,-0.263511,0.362920,0.037765,0.023824,-0.034627,-0.033959,0.019762,0.141659,0.055346,-0.124932
3,False,-0.992462,1.406792,0.032266,1.048060,-0.442415,0.022493,1.237495,-0.708838,-0.285095,...,-0.457199,0.179074,-0.747485,0.381123,-0.111548,0.108816,-0.436770,0.160321,0.130380,0.745097
4,False,0.594239,-1.013507,0.983505,-0.719509,0.618919,0.527497,-0.450578,0.074845,0.596864,...,-0.196619,0.485363,0.092997,-0.130889,-0.002419,0.195094,-0.120048,-0.031165,0.099765,-0.084680


In [70]:
## Agora sabemos que devemos usar PC0, PC3, PC1 e PC13 para fazer nossas predições
    # Treinando o modelo de Logistic Regression
vars_selecionadas = vars_atuais[:-1]

X = df_final[vars_selecionadas]
y = df_final[['class']]

logreg = linear_model.LogisticRegression()
logreg.fit(X, y)

c:\Users\jpzam\anaconda3\lib\site-packages\sklearn\utils\validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LogisticRegression()

Com o modelo treinado, tudo que faltava era aplicar o treinamento no dataset 'air_system_present_year.csv'

In [71]:
# Importing the current .csv data
df_present = pd.read_csv('C:/Users/jpzam/Desktop/Case_study_selection/air_system_present_year.csv')

print(df_present.shape)
df_present.head()

(16000, 171)


,class,aa_000,ab_000,ac_000,ad_000,ae_000,af_000,ag_000,ag_001,ag_002,...,ee_002,ee_003,ee_004,ee_005,ee_006,ee_007,ee_008,ee_009,ef_000,eg_000
0,neg,60,0,20,12,0,0,0,0,0,...,1098,138,412,654,78,88,0,0,0,0
1,neg,82,0,68,40,0,0,0,0,0,...,1068,276,1620,116,86,462,0,0,0,0
2,neg,66002,2,212,112,0,0,0,0,0,...,495076,380368,440134,269556,1315022,153680,516,0,0,0
3,neg,59816,na,1010,936,0,0,0,0,0,...,540820,243270,483302,485332,431376,210074,281662,3232,0,0
4,neg,1814,na,156,140,0,0,0,0,0,...,7646,4144,18466,49782,3176,482,76,0,0,0


In [72]:
# Limpando o dataset 'present', que estava bem organizado, para falar a verdade.
df_present = df_present.replace('na', np.nan)

for col in df_present.columns:
    if df_present[col].isna().sum() > 0:
        moda = df_present[col].mode()[0]
        df_present[col] = df_present[col].fillna(moda)

df_present.iloc[:, 1:] = df_present.iloc[:, 1:].astype(float)

df_present['class'] = df_present['class'] != 'neg' 

df_present_filtered = df_present[['class'] + vif_features]

C:\Users\jpzam\AppData\Local\Temp\ipykernel_10920\417251192.py:9: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df_present.iloc[:, 1:] = df_present.iloc[:, 1:].astype(float)


In [73]:
df_present_filtered.head()

,class,ab_000,ac_000,ae_000,af_000,ag_000,ai_000,aj_000,ak_000,ar_000,...,dk_000,dq_000,dr_000,dx_000,dy_000,dz_000,ea_000,eb_000,ef_000,eg_000
0,False,0.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1100.0,574.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0
1,False,0.0,68.0,0.0,0.0,0.0,0.0,60.0,0.0,0.0,...,0.0,3996.0,584.0,6368.0,36.0,0.0,0.0,0.0,0.0,0.0
2,False,2.0,212.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,10262714.0,1278664.0,4434614.0,70900.0,0.0,0.0,26002880.0,0.0,0.0
3,False,0.0,1010.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1179900.0,0.0,0.0
4,False,0.0,156.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,13664.0,110.0,0.0,0.0,813740.0,0.0,0.0


In [74]:
## Aplicando o mesmo scaler
df_present_std = scaler.transform(df_present_filtered)

## E a mesma transformação de PCA
pc_present = pca.transform(df_present_std)

present_final = pd.DataFrame(pc_present, columns = [f'PC{n}' for n in range (0,30)])

present_final = pd.concat([df_present_filtered['class'], present_final], axis = 1)

present_final

,class,PC0,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,...,PC20,PC21,PC22,PC23,PC24,PC25,PC26,PC27,PC28,PC29
0,False,-0.954508,0.467224,-0.469084,0.300431,-0.219167,-0.199880,0.199176,0.007791,-0.204745,...,-0.215631,0.248439,-0.038605,0.124546,-0.001187,0.049185,-0.063643,-0.148174,0.003703,0.046420
1,False,-1.092516,0.768575,-0.789453,0.416885,-0.333607,-0.359894,0.381620,-0.015500,-0.453898,...,-0.223612,0.181479,-0.036495,0.088703,0.033784,0.033261,-0.068579,-0.055241,-0.027800,0.004783
2,False,2.030254,-0.360344,0.963459,-1.052584,0.555658,0.407001,-0.744306,-0.210029,1.110923,...,0.116090,0.193959,0.645825,-0.512211,0.148319,1.115087,0.245333,-0.076778,0.146829,-0.603064
3,False,0.189389,-0.883536,0.824437,-0.553460,0.490946,0.372418,-0.421884,0.017407,0.416241,...,-0.264663,0.594402,0.209758,-0.011809,0.111631,0.051785,0.329536,0.267278,-0.067334,0.222053
4,False,-1.074066,0.788236,-0.784649,0.390230,-0.313966,-0.353220,0.356597,-0.014396,-0.412834,...,-0.227453,0.182062,-0.054300,0.094535,0.024500,0.009592,-0.039020,-0.023301,-0.019355,-0.011467
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15995,False,1.590288,-0.429024,0.814270,-0.844809,0.704878,0.535055,-0.618694,0.162863,0.918803,...,0.504029,-1.547958,-0.193917,-0.317707,0.494603,-0.351935,0.367931,-0.099625,0.025263,-0.655635
15996,False,-1.100062,0.880192,-0.786965,0.459113,-0.345881,-0.421068,0.390406,0.041324,-0.547391,...,-0.220940,0.172972,-0.036071,0.082251,0.027842,0.021439,-0.075707,-0.023234,-0.024616,0.005912
15997,False,0.774154,-1.129640,1.012499,-0.889791,1.036816,0.695333,0.010739,0.438029,0.440792,...,-0.071330,0.644794,0.130807,-0.104972,0.282260,0.087553,0.454485,-0.063924,-0.125134,0.157160
15998,False,-1.022022,0.638349,-0.651930,0.363831,-0.281838,-0.293387,0.305158,-0.006237,-0.347707,...,-0.223858,0.204941,-0.040516,0.105470,0.013067,0.043914,-0.064606,-0.094197,-0.020991,0.011254


In [77]:
## Por fim, fazendo as predições.
present_X = present_final[vars_selecionadas]

predictions = logreg.predict_proba(present_X)

print(predictions[:,1])

[0.00333695 0.00280577 0.00633282 ... 0.00750145 0.00302862 0.00268421]


Com isso, o modelo está encerrado. 

É claro que, para confirmação, poderíamos calcular a quantidade de falsos-positivos, falsos-negativos etc. e, disso, derivar uma métrica de acurácia do modelo.

Com o modelo feito, a 'empresa' da questão poderia, agora, predizer quais dos caminhões devem vir a dar defeito e enviá-los para manutenção antecipadamente, evitando custos extras de resgate, por exemplo